<a href="https://colab.research.google.com/github/siwarnasri/MlOps_CustomerSatisfaction/blob/main/2_1_Experiment_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.1 - Experiment Tracking with MLflow / W&B

When training models, you often need to run hundreds of experiments with different model types and different hyperparameters to find out what works best. Tracking each experiment and how each design decision affects model performance is almost infeasible without additional tools. Therefore, experiment trackers such as [TensorBoard](https://www.tensorflow.org/tensorboard), [Weights & Biases](https://wandb.ai/site), or [MLflow](https://mlflow.org/) are often one of the first touch points ML practitioners have with MLOps when exploring ML. In addition, these tools are invaluable for larger ML teams, as they allow them to share experimental results and collaborate during experimentation.

Since there are many excellent tools for tracking experiments, we should try to avoid lock-in to a particular vendor by writing modular ML code that allows us to easily switch between different tools. ZenML will do just that for us.

This lesson is about how to effectively track experiments using [MLflow] (https://mlflow.org/), one of the most popular open-source MLOps tools used by many ML teams in practice.

For research-heavy ML teams, we have also included a short bonus section at the end describing how to use [Weights & Biases] (https://wandb.ai/site) instead of MLflow in your ZenML pipelines.

Please run the following commands to install both tools with their respective dependencies. This will also reboot your notebook's kernel.

In [ ]:
%pip install "zenml[server]"
!zenml integration install sklearn mlflow wandb -y
!rm -rf .zen
!zenml init
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.0 MB/s eta 0:

NumExpr defaulting to 2 threads.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.6 MB/s eta 0:00:00
⠦ Installing integrations...  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 26.3 MB/s eta 0:00:00
⠋ Installing integrations...  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

{'status': 'ok', 'restart': True}

**Note about Colab:** On Colab, you will need an [ngrok account] (https://dashboard.ngrok.com/signup) to view some of the visualizations later. Please set up an account and then set your user id below:

In [ ]:
NGROK_TOKEN = "3veC4w6NbwgWUWWfW5kuF_7tagYBcjcYZXissQQhK99"  # TODO: set your ngrok token if you are working on Colab

In [ ]:
from zenml.environment import Environment

if Environment.in_google_colab():  # Colab only setup

    IN_COLAB = True

    # clone zenbytes repo to get source code of previous lessons
    !git clone https://github.com/zenml-io/zenbytes.git  # noqa
    !mv zenbytes/steps .
    !mv zenbytes/pipelines .

    # install ngrok and expose port 8080
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

fatal: destination path 'zenbytes' already exists and is not an empty directory.
mv: cannot stat 'zenbytes/steps': No such file or directory
mv: cannot stat 'zenbytes/pipelines': No such file or directory
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Next, we import our pipeline definition and some of the pipeline steps we created in the previous lessons:

In [ ]:
from pipelines.digits_pipeline import digits_pipeline
from steps.evaluator import evaluator
from steps.importer import importer

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
The @pipeline decorator that you used to define your digits_pipeline pipeline is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your pipelines to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty.html
The @step decorator that you used to define your evaluatorstep is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your steps to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty
The @step decorator that you used to define your importerstep is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your steps to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation h

## Register an MLFlow Experiment Tracker

[MLflow](https://mlflow.org/) is an amazing open source MLOps platform that provides powerful tools for various ML lifecycle steps, such as experiment tracking, code packaging, model deployment, and more. In this lesson, we will focus on the [MLflow Tracking](https://mlflow.org/docs/latest/tracking.html) component, but we will learn about other MLflow features in later lessons.

To use MLFlow in a ZenML pipeline, we first need to add MLflow to our ZenML MLOps stack.
To do this, we register a new experiment tracker with ZenML, which we then add to our current stack:

In [ ]:
# Register the MLflow experiment tracker
!zenml experiment-tracker register mlflow_tracker --flavor=mlflow

# Create a new stack that includes an MLflow experiment
!zenml stack register mlflow_exp_tracker_stack -a default -o default -e mlflow_tracker

# Set the new stack as active
!zenml stack set mlflow_exp_tracker_stack

NumExpr defaulting to 2 threads.
⠋ Registering experiment tracker 'mlflow_tracker'...
⠙ Registering experiment tracker 'mlflow_tracker'...
⠹ Registering experiment tracker 'mlflow_tracker'...
⠸ Registering experiment tracker 'mlflow_tracker'...
⠼ Registering experiment tracker 'mlflow_tracker'...
⠴ Registering experiment tracker 'mlflow_tracker'...
⠦ Registering experiment tracker 'mlflow_tracker'...
⠧ Registering experiment tracker 'mlflow_tracker'...
⠇ Registering experiment tracker 'mlflow_tracker'...
⠏ Registering experiment tracker 'mlflow_tracker'...
⠋ Registering experiment tracker 'mlflow_tracker'...
⠙ Registering experiment tracker 'mlflow_tracker'...
⠹ Registering experiment tracker 'mlflow_tracker'...
⠸ Registering experiment tracker 'mlflow_tracker'...
⠼ Registering experiment tracker 'mlflow_tracker'...
⠴ Registering experiment tracker 'mlflow_tracker'...
⠧ Registering experiment tracker 'mlflow_tracker'...
⠇ Registering experiment tracker 'mlflow_tracker'...
⠏ Registering

Let's also turn off MLflow warnings:

In [ ]:
from absl import logging as absl_logging
import warnings

warnings.filterwarnings("ignore")
absl_logging.set_verbosity(-10000)

## Using MLFlow in a ZenML pipeline

To integrate the MLFlow experiment tracker into our previously defined ZenML pipeline, we only need to change the `svc_trainer` step. We define a new step `svc_trainer_mlflow` where we use the MLflow function [mlflow.sklearn.autolog()`] (https://www.mlflow.org/docs/latest/python_api/mlflow.sklearn.html#mlflow.sklearn.autolog) to automatically log all relevant attributes and metrics of our model into MLflow.

By adding the parameter `experiment_tracker=mlflow_tracker` in the `@step` decorator, ZenML automatically takes care of initializing MLflow.

In [ ]:
import mlflow
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from zenml.steps import step

@step(enable_cache=False, experiment_tracker="mlflow_tracker")
def svc_trainer_mlflow(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn SVC classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
    model = SVC(gamma=1e-3)
    model.fit(X_train, y_train)
    return model

svc_mlflow_pipeline = digits_pipeline(
    importer=importer(),
    trainer=svc_trainer_mlflow(),
    evaluator=evaluator(),
)

The @step decorator that you used to define your svc_trainer_mlflowstep is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your steps to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.


Now, let's do the same for our decision tree trainer step:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

@step(enable_cache=False, experiment_tracker="mlflow_tracker")
def tree_trainer_with_mlflow(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn decision tree classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    return model

tree_mlflow_pipeline = digits_pipeline(
    importer=importer(),
    trainer=tree_trainer_with_mlflow(),
    evaluator=evaluator(),
)

The @step decorator that you used to define your tree_trainer_with_mlflowstep is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your steps to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.


And that's it, we're all set up! Now all `pipeline.run()` calls will automatically log all hyperparameters and metrics to MLflow. Let's try it out:

In [ ]:
svc_mlflow_pipeline.run(unlisted=True)
tree_mlflow_pipeline.run(unlisted=True)

Initiating a new run for the pipeline: digits_pipeline.
Executing a new run.
Using user: default
Using stack: mlflow_exp_tracker_stack
  artifact_store: default
  orchestrator: default
  experiment_tracker: mlflow_tracker
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.
Step importer has started.
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.
Using the Out

To compare our runs within the MLflow user interface, run the following cell and then open http://127.0.0.1:4997/ in your browser.

In [ ]:
# This will start a serving process for mlflow
#  - if you want to continue in the notebook you need to manually
#  interrupt the kernel

from zenml.environment import Environment
from zenml.integrations.mlflow.mlflow_utils import get_tracking_uri


def open_mlflow_ui(port=4997):

    if Environment.in_google_colab():  # Colab only setup
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")

    !mlflow ui --backend-store-uri="{get_tracking_uri()}" --port={port}


open_mlflow_ui()

INFO:pyngrok.ngrok:Opening tunnel named: http-4997-f0016a2a-f9c6-4276-8e07-92a167edd011


Opening tunnel named: http-4997-f0016a2a-f9c6-4276-8e07-92a167edd011


INFO:pyngrok.process.ngrok:t=2023-10-03T04:45:55+0000 lvl=info msg="no configuration paths supplied"


t=2023-10-03T04:45:55+0000 lvl=info msg="no configuration paths supplied"


t=2023-10-03T04:45:55+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml


...

t=2023-10-03T04:45:56+0000 lvl=info msg=start pg=/api/tunnels id=fd90369f44c42d59


INFO:pyngrok.process.ngrok:t=2023-10-03T04:45:56+0000 lvl=info msg="started tunnel" obj=tunnels name=http-4997-f0016a2a-f9c6-4276-8e07-92a167edd011 addr=http://localhost:4997 url=https://b09e-104-197-53-233.ngrok-free.app


In Colab, use this URL instead: NgrokTunnel: "https://b09e-104-197-53-233.ngrok-free.app" -> "http://localhost:4997"!
t=2023-10-03T04:45:56+0000 lvl=info msg="started tunnel" obj=tunnels name=http-4997-f0016a2a-f9c6-4276-8e07-92a167edd011 addr=http://localhost:4997 url=https://b09e-104-197-53-233.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2023-10-03T04:45:56+0000 lvl=info msg=end pg=/api/tunnels id=fd90369f44c42d59 status=201 dur=40.896716ms


t=2023-10-03T04:45:56+0000 lvl=info msg=end pg=/api/tunnels id=fd90369f44c42d59 status=201 dur=40.896716ms
[2023-10-03 04:45:58 +0000] [3954] [INFO] Starting gunicorn 21.2.0
[2023-10-03 04:45:58 +0000] [3954] [INFO] Listening at: http://127.0.0.1:4997 (3954)
[2023-10-03 04:45:58 +0000] [3954] [INFO] Using worker: sync
[2023-10-03 04:45:58 +0000] [3959] [INFO] Booting worker with pid: 3959
[2023-10-03 04:45:58 +0000] [3960] [INFO] Booting worker with pid: 3960
[2023-10-03 04:45:58 +0000] [3961] [INFO] Booting worker with pid: 3961
[2023-10-03 04:45:58 +0000] [3962] [INFO] Booting worker with pid: 3962


First, get an overview of all your runs, as shown below:

![MLflow UI](_assets/2-1/mlflow_ui.png)

Click on the `Parameters >` tab at the top of the table to see *all* of your model's hyperparameters. Now you can see at a glance which model performed best and which hyperparameters changed between runs. In our case, we can see that the SVC model with `gamma=0.001` achieved the best test accuracy of `0.969`.

If we click on one of the links in the `Start Time` column, we can see additional details about that particular run. In particular, under the "Artifacts" tab, we can find a "model.pkl" file that we can now use to deploy our model in an inference/production environment. In the next lesson, `2-2_Local_Deployment.ipynb`, we will learn how to do this automatically as part of our pipelines using the [MLflow Models](https://mlflow.org/docs/latest/models.html) component.

If you want to manually view the MLflow logs for your runs, you can
can specify the logging location using the `experiment_tracker_url` metadata field of the
Trainer step of your pipeline run, for example:

In [ ]:
from zenml.post_execution import get_unlisted_runs

pipeline_run = get_unlisted_runs()[0]
step = pipeline_run.get_step("trainer")
experiment_tracker_url = step.metadata["experiment_tracker_url"].value

print(experiment_tracker_url)

## Alternative tool: weights and biases

Of course, MLflow is not the only tool you can use for experiment tracking. The following example shows how to do the same thing with another experiment tracking tool: [Weights & Biases](https://wandb.ai/site).
This example assumes that you already have some familiarity with W&B. In particular, you will need a Weights & Biases account, which you can set up for free [here](https://wandb.ai/login?signup=true).

You will then need to define the following three variables to authorize yourself in W&B and to tell ZenML which entity/project you want to log in to:
- `WANDB_API_KEY`: your API key, which you can get from [https://wandb.ai/authorize](https://wandb.ai/authorize). Make sure you never share this key (in particular, remove the key before putting the notebook in public Git repositories).
- `WANDB_ENTITY`: the entity (team or user) that owns the project you want to log in to. If you are only using W&B, just enter your username here.
- `WANDB_PROJECT`: the name of the W&B project you want to log in to. If you have never used W&B or want to start a new project, just enter the new project name here, e.g. `MLOps Notebooks`.

In [ ]:
WANDB_API_KEY = None  # TODO: replace this with your W&B API key
WANDB_ENTITY = None  # TODO: replace this with your W&B entity name
WANDB_PROJECT = "zenbytes"  # TODO: replace this with your W&B project name (if you want to log to a specific project)

In [ ]:
# Register the W&B experiment tracker
!zenml experiment-tracker register wandb_tracker --flavor=wandb --api_key={WANDB_API_KEY} --entity={WANDB_ENTITY} --project_name={WANDB_PROJECT}

# Create a new stack that includes a W&B experiment tracker
!zenml stack register wandb_stack -a default -o default -e wandb_tracker

# Set the new stack as active
!zenml stack set wandb_stack

Next, we need to add wandb to our `svc_trainer` step and use it to initialize our ZenML pipeline. The overall structure is the same as in the MLflow example above: We simply add a `@enable_wandb` decorator to our step, and then we can use the `wandb` functionality within the step as we see fit.

The main difference from the previous MLflow example is that W&B does not have the sklearn autolog function. Instead, we need to call `wandb.log(...)` for each value we want to log in Weights & Biases.

Since we also want to log our test result, we also need to modify our `evaluator` step accordingly.

Also, when wandb is used in different steps within a pipeline, ZenML initializes wandb and makes sure that the experiment name matches the pipeline name and the experiment run name matches the pipeline run name. This creates a link between the pipelines in ZenML and the experiments in wandb.

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
import wandb
from zenml.steps import step

from pipelines.digits_pipeline import digits_pipeline
from steps.importer import importer


@step(enable_cache=False, experiment_tracker="wandb_tracker")
def svc_trainer_wandb(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn SVC classifier and log to W&B."""
    gamma = 1e-3
    wandb.log({"gamma": gamma})  # log gamma hparam to wandb
    model = SVC(gamma=gamma)
    model.fit(X_train, y_train)
    return model

@step(enable_cache=False, experiment_tracker="wandb_tracker")
def evaluator_wandb(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
    """Calculate the accuracy on the test set and log to W&B."""
    test_acc = model.score(X_test, y_test)
    wandb.log({"test acc": test_acc})  # log test_acc to wandb
    print(f"Test accuracy: {test_acc}")
    return test_acc

svc_wandb_pipeline = digits_pipeline(
    importer=importer(),
    trainer=svc_trainer_wandb(),
    evaluator=evaluator_wandb(),
)

Finally, run the cell below to run your pipeline with different gamma values.

In [ ]:
svc_wandb_pipeline.run(unlisted=True)

Run the cell below and follow the link to see the run in your Weights & Biases project:

In [ ]:
trainer_step = get_unlisted_runs()[0].get_step("trainer")
experiment_tracker_url = trainer_step.metadata["experiment_tracker_url"].value
print(experiment_tracker_url)

For a more detailed example of using Weights & Biases to track experiments in your ZenML pipeline, see the [ZenML wandb_tracking example](https://github.com/zenml-io/zenml/tree/main/examples/wandb_tracking).

In the [next notebook](2-2_Local_Deployment.ipynb) we will learn how to deploy models locally using MLflow.